# Export intra- and inter-rater disagreements for Adobe Premiere markers

This notebook generates CSV files for each trial that had disagreement above the thresholds and that
did not have uncertainty. 

Before running this, run data through the algorithm, with the tabulated results in the
`data/results` folder.

After running this, in Adobe Premiere Pro run the script `NIMR import disagreement to markers.xml` to
import these as sequence markers. For more, see `ROxi breath marking in Adobe Premiere.pptx`.

In [8]:
import pandas as pd
import numpy as np

In [9]:
%load_ext autoreload
%autoreload 2

# only display 2 decimals
pd.set_option('display.precision', 2)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# Inputs and settings

# The features.csv of results. Be sure to use a 5 s window (not 30 s as we use when evaluating the algorithm).
filenames = ['nimr_Tr1__wsize-5_winc-5_batch-1-final2_20250614-101602_features.csv']

export_dir = '../data/3ps/disagreement markers/1 exported from Python'

# These must be sorted from most to least
disagreement_thresholds = {
    'lower': [12, 6, 3],
    'color': ['Red', 'Orange', 'Yellow']
}
uncert_color = 'White'

metric_base = 'RR ref disagreement '
metrics = {
    # Keys are the Python DF column names; Values are the filenames for Premiere Pro to match sequence names
    'panel':     ' Group comparison', 
    'panelist D':'Daniel comparison',
    'panelist J':'Jason comparison',
    'panelist O':'Oliver comparison',
}
metric_suffix = ' (bpm)'

uncert_base = 'RR uncertainty '
uncert_suffix = ' (mean)'

In [11]:
# Read data file(s)

if len(filenames) == 1:
    frames = pd.read_csv(f"../data/results/{filenames[0]}")
else:
    dfs = list(pd.DataFrame())
    for filename in filenames:
        dfs.append(pd.read_csv(f"../data/results/{filename}", low_memory=False))

    frames = pd.concat(dfs).copy()      # The copy() avoids fragmentation warnings later.

In [12]:
# In each row of the frames DF, for each of the metrics, assign a marker color:
#  - A non-zero uncertainty value gets a white marker.
#  - Disagreement less than the lowest tier gets no color. 

marker_colors = [uncert_color] + disagreement_thresholds['color']

for metric in metrics.keys():
    metric_full = metric_base + metric + metric_suffix
    uncert_full = uncert_base + metric + uncert_suffix
    conditions = [(frames[uncert_full] > 0)] + [
        (frames[metric_full] >= disagreement_thresholds['lower'][i]) for i in range(len(disagreement_thresholds['lower']))
    ]

    frames[f'Marker Color.{metric}'] = np.select(conditions, marker_colors, default='')

frames

,trial-frame,index,frame index,time start,time end,RR uncertainty panel (mean),RR uncertainty panelist D (mean),RR uncertainty panelist J (mean),RR uncertainty panelist O (mean),avg rr ref panelist,...,quality - psd n valid peaks-product of all PSD,quality - template matching-mean loc nonconforming pulses,quality - template matching-pct amplitude outliers,quality - template matching-pct diagnostic quality pulses,quality - template matching-pct good pulse shapes,quality - template matching-pct poor pulse shapes,Marker Color.panel,Marker Color.panelist D,Marker Color.panelist J,Marker Color.panelist O
0,0-0,0,0,0,5,0.27,0.0,0.0,0.80,defaultdict(<function <lambda> at 0x31fb21000>...,...,0.0,0.4166666666666667,0.00,0.00,0.00,1.00,White,,,White
1,0-1,1,1,5,10,0.33,0.0,0.0,1.00,defaultdict(<function <lambda> at 0x31fb21000>...,...,1.0,0.42857142857142855,0.14,0.00,0.14,0.86,White,,,White
2,0-2,2,2,10,15,0.13,0.0,0.0,0.38,defaultdict(<function <lambda> at 0x31fb21000>...,...,NaN,NaN,NaN,NaN,NaN,NaN,White,,,White
3,0-3,3,3,15,20,0.00,0.0,0.0,0.00,defaultdict(<function <lambda> at 0x31fb21000>...,...,1.0,0.5,0.18,0.09,0.09,0.91,,,,
4,0-4,4,4,20,25,0.00,0.0,0.0,0.00,defaultdict(<function <lambda> at 0x31fb21000>...,...,1.0,0.42424242424242425,0.18,0.18,0.18,0.82,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3595,99-31,31,31,155,160,0.00,0.0,0.0,0.00,defaultdict(<function <lambda> at 0x31fb21000>...,...,1.0,0.4444444444444444,0.22,0.00,0.11,0.89,Red,,,
3596,99-32,32,32,160,165,0.00,0.0,0.0,0.00,defaultdict(<function <lambda> at 0x31fb21000>...,...,0.0,0.4375,0.25,0.00,0.00,1.00,Orange,,,
3597,99-33,33,33,165,170,0.00,0.0,0.0,0.00,defaultdict(<function <lambda> at 0x31fb21000>...,...,1.0,0.4166666666666667,0.17,0.00,0.00,1.00,Red,,,
3598,99-34,34,34,170,175,0.00,0.0,0.0,0.00,defaultdict(<function <lambda> at 0x31fb21000>...,...,0.0,0.34375,0.00,0.50,0.50,0.50,Red,,,


In [13]:
# Marker CSV structure

marker_cols_export = ['Marker Name', 'Description', 'In', 'Out', 'Duration', 'Marker Type', 'Marker Color']

In [14]:
# For each trial, for each metric, create a DataFrame by copying and renaming the relevant columns,
# with "excluded" or the disagreement value assigned to "Marker Name", and export a CSV of the
# markers that have colors

print("Disagreement markers were exported for the following:")
for metric, metric_export in metrics.items():
    print()     # blank line between panelists
    metric_count = 0
    for subject_id in frames['subject id'].unique():
        metric_full = metric_base + metric + metric_suffix
        marker_color_col = f'Marker Color.{metric}'     # name of the column created earlier
        
        this_df = frames[(frames['subject id'] == subject_id) & (frames[marker_color_col] != '')].copy()
        this_dis_df = this_df[this_df[marker_color_col] != uncert_color]
        
        # If any disagreements, print this for the panelist's attention
        if len(this_dis_df) > 0:
            print(f"{subject_id} {metric_export}")
            metric_count += 1
        # Even if not any disagreements, write a CSV so that Premiere will delete any prior
        # disagreement markers
        this_df.rename(columns={
            'time start':               'In',
            'time end':                 'Out',
            f'Marker Color.{metric}':   'Marker Color',
        }, inplace=True)
        this_df['Marker Name'] = this_df[metric_full].apply(lambda x: f'{x:.1f} bpm dis')
        this_df.loc[(this_df['Marker Color'] == uncert_color), 'Marker Name'] = 'excluded'
        this_df['Description'] = ''
        this_df['Duration'] = this_df['Out'] - this_df['In']
        this_df['Marker Type'] = 'Comment'
        # Drop unnecessary columns
        this_df = this_df[marker_cols_export].copy()

        # Merge consecutive "exclude" markers into one for a cleaner Premiere look
        merged_rows = []
        current_block = None

        for _, row in this_df.iterrows():
            if row["Marker Name"] == "excluded":
                if (
                    current_block is not None and
                    current_block["Marker Name"] == "excluded" and
                    row["In"] == current_block["Out"]
                ):
                    # Extend the current block
                    current_block["Out"] = row["Out"]
                    current_block["Duration"] = current_block["Out"] - current_block["In"]
                else:
                    # Finish previous block and start new one
                    if current_block is not None:
                        merged_rows.append(current_block)
                    current_block = row.copy()
            else:
                if current_block is not None:
                    merged_rows.append(current_block)
                    current_block = None
                merged_rows.append(row)

        # Add last block if it exists
        if current_block is not None:
            merged_rows.append(current_block)

        # Convert to DataFrame
        merged_df = pd.DataFrame(merged_rows, columns=marker_cols_export)

        merged_df.to_csv(f'{export_dir}/{subject_id} {metric_export}.csv', index=False)

    if metric_count:
        print(f"Count for {metric}: {metric_count} sessions")

Disagreement markers were exported for the following:

N07-017  Group comparison
N07-018  Group comparison
N07-019  Group comparison
N07-021  Group comparison
N07-022  Group comparison
N07-023  Group comparison
N07-024  Group comparison
N07-025  Group comparison
N07-027  Group comparison
N07-028  Group comparison
N07-029  Group comparison
N07-030  Group comparison
N07-032  Group comparison
N07-033  Group comparison
N07-034  Group comparison
N07-035  Group comparison
N09-034  Group comparison
N09-035  Group comparison
N09-036  Group comparison
N09-037  Group comparison
N09-038  Group comparison
N09-039  Group comparison
N09-041  Group comparison
N09-042  Group comparison
N09-045  Group comparison
N09-046  Group comparison
N09-048  Group comparison
N09-050  Group comparison
N09-051  Group comparison
N09-052  Group comparison
N09-053  Group comparison
N09-055  Group comparison
N09-056  Group comparison
N09-057  Group comparison
N09-058  Group comparison
N09-059  Group comparison
N09-060  